In [1]:
import cv2

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/home/piai/swimming/yolov5-env


In [3]:
import torch

In [10]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu122

Looking in indexes: https://download.pytorch.org/whl/cu122
ERROR: Could not find a version that satisfies the requirement torchaudio (from versions: none)
ERROR: No matching distribution found for torchaudio
Note: you may need to restart the kernel to use updated packages.


In [11]:
!nvidia-smi

Wed Sep 18 16:20:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080        Off | 00000000:17:00.0 Off |                  N/A |
| 18%   37C    P8               2W / 215W |     10MiB /  8192MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
# %pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.91 🚀 Python-3.10.9 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080, 7974MiB)
Setup complete ✅ (8 CPUs, 62.5 GB RAM, 108.2/233.2 GB disk)


In [13]:
from ultralytics import YOLO
from IPython.display import display, Image

In [15]:
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  # clone repo

%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

%pip install ipywidgets


%pip install --upgrade imageio
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 16954, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 16954 (delta 76), reused 96 (delta 46), pack-reused 16805 (from 1)
Receiving objects: 100% (16954/16954), 15.70 MiB | 22.61 MiB/s, done.
Resolving deltas: 100% (11605/11605), done.
/home/piai/swimming/yolov5-env/yolov5


/home/piai/swimming/yolov5-env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Setup complete. Using torch 2.4.1+cu121 (NVIDIA GeForce RTX 2080)


In [16]:
%pip install PyYAML

import yaml
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="X8J8YI9RmLOJswz3U0UI")
project = rf.workspace("se-zgnpx").project("swimming-drowning-2")
version = project.version(1)
dataset = version.download("yolov5")

Note: you may need to restart the kernel to use updated packages.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Swimming-Drowning-2-1 in yolov5pytorch:: 100%|██████████| 29590/29590 [00:02<00:00, 10391.53it/s]


In [21]:
%pip install pyyaml

Note: you may need to restart the kernel to use updated packages.


In [22]:
%cat /home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/data.yaml

names:
- Drowning
- Person out of water
- Swimming
nc: 3
roboflow:
  license: CC BY 4.0
  project: swimming-drowning-2
  url: https://universe.roboflow.com/se-zgnpx/swimming-drowning-2/dataset/1
  version: 1
  workspace: se-zgnpx
test: ../test/images
train: Swimming-Drowning-2-1/train/images
val: Swimming-Drowning-2-1/valid/images


In [23]:
# 사용할 데이터셋 경로 및 데이터셋의 yaml 파일 경로 지정
data_dir = '/home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1'
data_yaml = '/home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/data.yaml'

In [24]:
import yaml
from IPython.display import display

# 데이터셋 yaml 파일 확인
with open(data_yaml) as f:
    film = yaml.load(f, Loader=yaml.FullLoader)
    display(film)

{'names': ['Drowning', 'Person out of water', 'Swimming'],
 'nc': 3,
 'roboflow': {'license': 'CC BY 4.0',
  'project': 'swimming-drowning-2',
  'url': 'https://universe.roboflow.com/se-zgnpx/swimming-drowning-2/dataset/1',
  'version': 1,
  'workspace': 'se-zgnpx'},
 'test': '../test/images',
 'train': 'Swimming-Drowning-2-1/train/images',
 'val': 'Swimming-Drowning-2-1/valid/images'}

In [25]:
# yaml 파일의 train, val 데이터가 있는 경로 수정 (기존 경로 -> 구글 드라이브에 저장된 경로로)
film['train'] = '/home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/train/images'
film['val'] = '/home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/valid/images'

with open(data_yaml, 'w') as f:
    yaml.dump(film, f)

print('변경된 yaml 파일 :')
with open(data_yaml) as f:
    film = yaml.load(f, Loader=yaml.FullLoader)
    display(film)

변경된 yaml 파일 :


{'names': ['Drowning', 'Person out of water', 'Swimming'],
 'nc': 3,
 'roboflow': {'license': 'CC BY 4.0',
  'project': 'swimming-drowning-2',
  'url': 'https://universe.roboflow.com/se-zgnpx/swimming-drowning-2/dataset/1',
  'version': 1,
  'workspace': 'se-zgnpx'},
 'test': '../test/images',
 'train': '/home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/train/images',
 'val': '/home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/valid/images'}

In [26]:
%pip install wandb

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 33.0 MB/s eta 0:00:00a 0:00:01
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Note: you may need to restart the kernel to use updated packages.


In [27]:
import os

# 작업 디렉터리로 이동
os.chdir("/home/piai/swimming/yolov5-env/yolov5")

!python train.py --img 416 --batch 32 --epochs 100 --data /home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/data.yaml --weights yolov5n.pt 

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-09-18 16:34:52.855832: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-18 16:34:52.875912: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-18 16:34:52.882065: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visual

In [28]:
# 작업 디렉터리로 이동
os.chdir("/home/piai/swimming/yolov5-env/yolov5")

!python val.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --data /home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/data.yaml

val: data=/home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=416, conf_thres=0.1, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
WARNING ⚠️ confidence threshold 0.1 > 0.001 produces invalid results
YOLOv5 🚀 v7.0-366-gf7322921 Python-3.10.9 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080, 7974MiB)

Fusing layers... 
Model summary: 157 layers, 1763224 parameters, 0 gradients, 4.1 GFLOPs
val: Scanning /home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/valid/
                 Class     Images  Instances          P          R      mAP50   
                   all       1255       1958      0.855      0.846      0.883      0.576
              Drowning       1255       1180      0.874      0.901      0.928   

In [29]:
os.chdir("/home/piai/swimming/yolov5-env/yolov5")

In [30]:
!python detect.py --weights runs/train/exp/weights/best.pt --source //home/piai/SWIM.mp4 --img 640 --conf 0.25

detect: weights=['runs/train/exp/weights/best.pt'], source=//home/piai/SWIM.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-366-gf7322921 Python-3.10.9 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080, 7974MiB)

Fusing layers... 
Model summary: 157 layers, 1763224 parameters, 0 gradients, 4.1 GFLOPs
video 1/1 (1/10773) /home/piai/SWIM.mp4: 480x640 (no detections), 33.9ms
video 1/1 (2/10773) /home/piai/SWIM.mp4: 480x640 (no detections), 5.6ms
video 1/1 (3/10773) /home/piai/SWIM.mp4: 480x640 (no detections), 5.7ms
video 1/1 (4/10773) /home/piai/SWIM.mp4: 480x640 (no detections), 5.5ms
video 1/1 (5

In [31]:
import os

# 작업 디렉터리로 이동
os.chdir("/home/piai/swimming/yolov5-env/yolov5")

!python train.py --img 416 --batch 32 --epochs 200 --data /home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/data.yaml --weights yolov5n.pt 

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-09-18 19:31:23.004397: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-18 19:31:23.026018: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-18 19:31:23.032374: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visual

In [32]:
# 작업 디렉터리로 이동
os.chdir("/home/piai/swimming/yolov5-env/yolov5")

!python val.py --weights runs/train/exp/weights/best.pt --img 416 --conf 0.1 --data /home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/data.yaml

val: data=/home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/data.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=416, conf_thres=0.1, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
WARNING ⚠️ confidence threshold 0.1 > 0.001 produces invalid results
YOLOv5 🚀 v7.0-366-gf7322921 Python-3.10.9 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080, 7974MiB)

Fusing layers... 
Model summary: 157 layers, 1763224 parameters, 0 gradients, 4.1 GFLOPs
val: Scanning /home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/valid/
                 Class     Images  Instances          P          R      mAP50   
                   all       1255       1958      0.855      0.846      0.883      0.576
              Drowning       1255       1180      0.874      0.901      0.928   

In [33]:
!python detect.py --weights runs/train/exp2/weights/best.pt --source //home/piai/SWIM.mp4 --img 640 --conf 0.25

detect: weights=['runs/train/exp2/weights/best.pt'], source=//home/piai/SWIM.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-366-gf7322921 Python-3.10.9 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080, 7974MiB)

Fusing layers... 
Model summary: 157 layers, 1763224 parameters, 0 gradients, 4.1 GFLOPs
video 1/1 (1/10773) /home/piai/SWIM.mp4: 480x640 (no detections), 39.8ms
video 1/1 (2/10773) /home/piai/SWIM.mp4: 480x640 (no detections), 5.2ms
video 1/1 (3/10773) /home/piai/SWIM.mp4: 480x640 (no detections), 5.3ms
video 1/1 (4/10773) /home/piai/SWIM.mp4: 480x640 (no detections), 5.4ms
video 1/1 (

In [34]:
!python detect.py --weights runs/train/exp2/weights/best.pt --source //home/piai/익사2.mp4 --img 640 --conf 0.25

detect: weights=['runs/train/exp2/weights/best.pt'], source=//home/piai/익사2.mp4, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-366-gf7322921 Python-3.10.9 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080, 7974MiB)

Fusing layers... 
Model summary: 157 layers, 1763224 parameters, 0 gradients, 4.1 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
video 1/1 (1/301) /home/piai/익사2.mp4: 352x640 1 Drowning, 38.8ms
video 1/1 (2/301) /home/piai/익사2.mp4: 352x640 1 Drowning, 5.5ms
video 1/1 (3/301) /home/piai/익사2.mp4: 352x640 1 Drowning, 5.6ms
video 1/1 (4/301) /home/piai/익사2.mp4: 352x640 (no detections), 6.3

In [35]:
import os

# 작업 디렉터리로 이동
os.chdir("/home/piai/swimming/yolov5-env/yolov5")

!python train.py --img 416 --batch 32 --epochs 200 --data /home/piai/swimming/yolov5-env/yolov5/Swimming-Drowning-2-1/data.yaml --weights yolov5s.pt 

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-09-19 20:27:46.442454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 20:27:46.462359: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 20:27:46.468373: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visual